In [1]:
import os

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from xarray.coding.cftime_offsets import MonthEnd

In [2]:
home_dir = os.path.expanduser('~')
file_path = os.path.join(home_dir, 'Code/star-struck/data/download.grib')

In [3]:
def load_dataset(filename):
    """
    Load a dataset from a given file path.

    Parameters:
    - filename (str): The path to the dataset file.

    Returns:
    - xarray.Dataset: The loaded dataset with temperature in Celsius.
    """

    home_dir = os.path.expanduser('~')
    file_path = os.path.join(home_dir, 'Code/star-struck/data/', filename)

    ds = xr.open_dataset(file_path, engine='cfgrib')
    
    # convert Kelvin to Celcius
    ds['t2m'] -= 273.15

    return ds

In [4]:
ds = load_dataset(file_path)

In [5]:
print(ds)

<xarray.Dataset>
Dimensions:     (time: 32, step: 24, latitude: 70, longitude: 103)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2022-06-30 2022-07-01 ... 2022-07-31
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    surface     float64 ...
  * latitude    (latitude) float64 41.7 41.6 41.5 41.4 ... 35.1 35.0 34.9 34.8
  * longitude   (longitude) float64 19.37 19.47 19.57 ... 29.37 29.47 29.57
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    t2m         (time, step, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-08-13T23:49 GRIB to CDM+CF via cfgrib-0.9.1...


In [6]:
ds.t2m

<xarray.DataArray 't2m' (time: 32, step: 24, latitude: 70, longitude: 103)>
array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
...
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]]]], dtype=float32)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2022-06-30 2022-07-01 ... 2022-07-31
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    surface     float64 ...
  * latitude    (latitude) float64 41.7 41.6 41.5 41.4 ... 35.1 35.0 34.9 34.8
  * longitude   (longitude) float64 19.37 19.47 19.57 ... 29.37 29.47 29.57
    valid_time  (time, step) datetime64[ns] ...
Attributes: (12/30)
    GRIB_paramId:                             167
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      7210
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_shortName:                           2t
    GRIB_totalNumber:                         0
    GRIB_units:                               K
    long_name:                                2 metre temperature
    units:                                    K
    standard_name:                            unknown